In [ ]:
import aiohttp
import asyncio
import os
from dotenv import load_dotenv


# This is a very clear work in progress, just trying to move some code into a place where it can be cleaned later, but for now re-used.

load_dotenv()


PM_SETTINGS = {
    "BOT_TOKEN": "5a3556bb2de44a73ab2e5643cb633a6c",
    "THREAD_ID": "default",
    "DB_PATH": "../datasets/nba_sql.db",
    "LOCAL_HISTORY_DB": "sqlite+aiosqlite:///promptHistory.db",
    "VERBOSE": True,
    "openai_api_key": os.environ.get("OPENAI_API_KEY"),
}

PM_SETTINGS[
    "uri"
] = f"wss://www.approx.dev/ws/chat?thread_id={PM_SETTINGS['THREAD_ID']}"


async def get_gpt3_response(prompt, temperature=0, stop=None):
    if not PM_SETTINGS["openai_api_key"]:
        raise Exception("No OpenAI API key found")
    # print the prompt if verbose mode
    if PM_SETTINGS["VERBOSE"]:
        print(prompt)
    headers = {
        "Authorization": f"Bearer {PM_SETTINGS['openai_api_key']}",
        "Content-Type": "application/json",
    }
    data = {
        "prompt": prompt,
        "max_tokens": 500,
        "temperature": temperature,
        "model": "text-davinci-002",
        "presence_penalty": 0.4,
        "frequency_penalty": 0.4,
    }
    if stop:
        data["stop"] = stop
    async with aiohttp.ClientSession() as session:
        async with session.post(
            "https://api.openai.com/v1/completions", headers=headers, json=data
        ) as resp:
            answer = await resp.json()
    if "choices" in answer:
        return answer["choices"][0]["text"]
    else:
        print("Possible error: query returned:", answer)
    return answer.get("choices", [{"text": ""}])[0]["text"]


In [ ]:
await get_gpt3_response("Hello, world")

In [ ]:
prompts = [f"Hi, my name is {n} and I think science is" for n in ["Justin", "Jen", "Jill", "James"]]
res = await asyncio.gather(*[get_gpt3_response(p) for p in prompts])

In [ ]:
# 2.8 seconds in parallel


In [ ]:
res = [await get_gpt3_response(p) for p in prompts]

In [ ]:
# 9.9 seconds in serial! Parallel works.